# Equipo: Hercules Poirot
## Componentes
- Eva Mañans
- Mariano Puchades
- Antonio Herrera
- Justo Barco

### Enunciado
A crime has taken place and the detective needs your help. The detective gave you the crime scene report, but you somehow lost it. You vaguely remember that the crime was a **murder** that occurred sometime on **Jan.15, 2018** and that it took place in **SQL City**. Start by retrieving the corresponding crime scene report from the police department’s database.

In [1]:
import pandas as pd
import sqlite3
conn = sqlite3.connect("./Data/sql-murder-mystery.db")
cursor = conn.cursor()

In [2]:
def sql_query(query):
    cursor.execute(query)
    resultado = cursor.fetchall()
    col = [d[0] for d in cursor.description]
    df = pd.DataFrame(resultado, columns=col)
    return df

In [3]:
# Tomamos el informe de la policía que hace referencia a los datos facilitados
query = '''
SELECT *
FROM crime_scene_report
WHERE date = 20180115 and city = 'SQL City' and type = 'murder'
'''
df = sql_query(query)
df.description[0]

'Security footage shows that there were 2 witnesses. The first witness lives at the last house on "Northwestern Dr". The second witness, named Annabel, lives somewhere on "Franklin Ave".'

In [4]:
# Buscamos el testimonio del primer testigo, el que vive en la última casa de "Northwestern Dr"
query = '''
SELECT interview.transcript
FROM person
INNER JOIN interview
ON person.id = interview.person_id
WHERE address_street_name = "Northwestern Dr"
ORDER BY address_number DESC
LIMIT 1
'''
df = sql_query(query)
df.transcript[0]

'I heard a gunshot and then saw a man run out. He had a "Get Fit Now Gym" bag. The membership number on the bag started with "48Z". Only gold members have those bags. The man got into a car with a plate that included "H42W".'

In [9]:
# Buscamos a Annabel entre los testimonios 
query = '''
SELECT a.id, a.name, b.*
FROM interview AS b
INNER JOIN person AS a
ON a.id = b.person_id
WHERE a.name LIKE "%Annabel%" and a.address_street_name = "Franklin Ave"
'''
df = sql_query(query)
df.transcript[0]

'I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th.'

In [ ]:
# Buscamos a la persona que coincide con todas las pistas que nos dan ambos testimonios
query = '''
SELECT person.name
FROM get_fit_now_member
JOIN get_fit_now_check_in ON get_fit_now_member.id = get_fit_now_check_in.membership_id
JOIN person ON get_fit_now_member.person_id = person.id
JOIN drivers_license ON person.license_id = drivers_license.id
WHERE get_fit_now_member.membership_status = "gold"
    and get_fit_now_member.id LIKE "%48Z%"
    and get_fit_now_check_in.check_in_date = 20180109
    and drivers_license.plate_number LIKE "%H42W%";
'''
df = sql_query(query)
df.name[0]

'Jeremy Bowers'